# Create OWL (PTP) minimal topology

## Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fm = fablib_manager()
                     
    fm.show_config()
except Exception as e:
    print(f"Exception: {e}")

### List slices

In [ ]:
fm.list_slices()

## Choose sites

### Choose ALL PTP-enalbed sites

In [ ]:
sites = fm.get_site_names()
print(f"all sites: {len(sites)} ({sites})")

# ptp_sites = fm.list_sites(
#     output="list",
#     quiet=True,
#     filter_function = lambda x:x['ptp_capable'] is True and x['state'] == 'Active',
# )
# ptp_site_names = [x['name'] for x in ptp_sites]

# print(f"PTP-enabled and active sites: {len(ptp_site_names)} ({ptp_site_names})")


### If a certain number of sites (rather than ALL PTP-enabled sites) are desired

In [ ]:
ptp_site_names = fm.get_random_sites(count=2, filter_function=lambda x: x['ptp_capable'] is True and x['state'] == 'Active') 
print(f"Chosen PTP-enabled sites: {len(ptp_site_names)} ({ptp_site_names})")

## Create a slice with one node placed on each PTP-enabled site

The following creates a slice with one node on each PTP-capable site, with basic NICs connected to FABRIC's FABnetv4 internet. 

In [ ]:
slice_name = "test_slice"

In [ ]:
slice = fm.new_slice(name=slice_name)

for i, site_name in enumerate(ptp_site_names):
    node = slice.add_node(name=f"node{i}", site=site_name, image='docker_rocky_8')
    node.add_fabnet()
    print(f"adding node{i} at {site_name}")

#Submit Slice Request
slice.submit();

## (Optional) Observe the Slice's Attributes

In [ ]:
try:
    slice = fm.get_slice(name=slice_name)
    slice.show()
    slice.list_nodes()
    slice.list_networks()
    slice.list_interfaces()
except Exception as e:
    print(f"Exception: {e}")

## Check connectivity via Experimenter's network

The following checks Node0 --> each node in the topology

In [ ]:
node0 = slice.get_node(name="node0")
nodes = slice.get_nodes()

for i in range(len(nodes)):
    dst_node_addr = nodes[i].get_interface(network_name=f'FABNET_IPv4_{nodes[i].get_site()}').get_ip_addr()
    stdout, stderr = node0.execute(f'ping -c 3 {dst_node_addr}')

    
# except Exception as e:
#     print(f"Exception: {e}")

# Prepare each node for time precision experiments

## Install and setup linuxptp package  on nodes
Download the Ansible role to configure and install the LinuxPTP software. For more details regarding the steps performed in the playbook, please refer to the repo at [https://github.com/fabric-testbed/ptp](https://github.com/fabric-testbed/ptp)

In [ ]:
pre_requisites = None

# Set Deployment tool repository details
repo_branch = 'main'
repo_name = 'ptp'
destination_folder = f"""/tmp/{repo_name}-{repo_branch}"""
clone_instructions = f"""
cd /tmp/;rm -rf /tmp/{repo_name}-{repo_branch};git clone --branch {repo_branch} https://github.com/fabric-testbed/{repo_name}.git {destination_folder};
"""

### Setting PTP Install Restrictions (Leave it blank)

* If you do not want all interfaces synchronized to PTP, add the name of interfaces to avoid as shown
* Management interfaces are not considered and are avoided by default
* If you do not want the system clock synchronized to PTP set the 'SYNC_SYSTEM_CLOCK' to False
* If you do not have any restrictions for a node, you can omit that node from the list

Example:
```
NODE_RESTRICTIONS = { 
   'node1' : { 'AVOID_IFACES': ['enp6s0'],'SYNC_SYSTEM_CLOCK': False},
   'node2' : { 'AVOID_IFACES': ['enp6s0','enp7s0']},
}
```

In [ ]:
NODE_RESTRICTIONS = {}

### Restrict Ansible operation based on tags (Leave it blank)

* Possible values are ptp_stop,ptp_start,ptp_install 
* Only one tag is allowed
* If empty then all three are performed in the right sequence
* If NODE_RESTRICTIONS are applied along with the tags, the operations will not be performed on the AVOIDED INTERFACES

Example
```
ansible_tags = 'ptp_stop'
```

In [ ]:
ansible_tags = ''

### Run Ansible playbook on each node

In [ ]:
# Instruction to run ansible command from the node
ansible_instructions = f"""
cd {destination_folder}/ansible;ansible-playbook --connection=local --inventory 127.0.0.1, --limit 127.0.0.1 playbook_fabric_experiment_ptp.yml"""

#Create execute threads
execute_threads = {}

for node in nodes:
    if [ele for ele in ["rocky", "centos"] if (ele in node.get_image())]:
        pre_requisites = f"""
        sudo dnf -y install epel-release ; sudo dnf -y install ansible git;
        """
    elif [ele for ele in ["ubuntu", "debian"] if (ele in node.get_image())]:
        pre_requisites = f"""sudo apt-get update;sudo apt-get -y install ansible git;"""
    else:
        pre_requisites = None
    node_name = node.get_name()
    
    # Create JSON files for extra params that will be provided to ansible
    if node_name in NODE_RESTRICTIONS.keys():    
        extra_ansible_params = f""" --extra-vars @parameters.json""";
        with open('/tmp/'+node_name+'-parameters.json', 'w') as f:
            json.dump(NODE_RESTRICTIONS[node_name], f)
        print (f"Uploading install restrictions for {node_name}")    
        node.upload_file('/tmp/'+node_name+'-parameters.json',destination_folder+'/ansible/parameters.json')
    else:
        extra_ansible_params = ''
    if ansible_tags != '':
        extra_ansible_params = extra_ansible_params + ' --tags '+ansible_tags
        
    print (f"Running the PTP Deployment Ansible Playbook on {node.get_name()}")
    execute_threads[node] = node.execute_thread(\
                f"{pre_requisites}"\
                f"{clone_instructions}"\
                f"{ansible_instructions}"\
                f"{extra_ansible_params}",\
                output_file=f"/tmp/{node.get_name()}_ptpinstall.log"\
                )

    #Wait for results from threads
for node,thread in execute_threads.items():
    print(f"Waiting for result from node {node.get_name()}")
    stdout,stderr = thread.result()

print (f"Ansible Playbook run on all nodes completed\n")

## Start Docker and verify it is running

`docker ps` should print `CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES`

In [ ]:
for node in nodes:
    node.execute("sudo systemctl start docker")
    node.execute("sudo systemctl enable docker")
    node.execute("sudo usermod -aG docker rocky")
    
    print(f"\n Verify installtion... on {node.get_name()}")
    node.execute("docker ps")

# (Optional) Extend the slice (Add 14 days)

In [ ]:
try:
    slice = fm.get_slice(name=slice_name)
    print(f"Lease End         : {slice.get_lease_end()}")
       
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
import datetime

#Extend slice
end_date = (datetime.datetime.now().astimezone() + datetime.timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fm.get_slice(name=slice_name)
    slice.renew(end_date)
    print(f"New lease end date : {slice.get_lease_end()}")
    
except Exception as e:
    print(f"Exception: {e}")

# Delete the Slice

In [ ]:
# try:
#     slice = fm.get_slice(name=slice_name)
#     slice.delete()
# except Exception as e:
#     print(f"Exception: {e}")